<a href="https://colab.research.google.com/github/tteague19/xlab-security-course/blob/main/notebooks/05-defensive_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Defensive Distillation

The authors of [Distillation as a Defense to Adversarial
Perturbations against Deep Neural Networks](https://arxiv.org/pdf/1511.04508) give a description of the four key ideas behind distilling image classifiers as a defense against adversarial examples.

1. Start with hard labels (they describe this a series of one-hot vectors, but that is not necessarily how they would be stored in memory).
2. Train the initial model using a traditional procedure, but let the final layer have a softmax with a temperature greater than one.
3. Create a new training set using the outputs of this initial model. That is, instead of starting with hard labels like the previous model, we start with soft labels outputed by the initial model.
4. Train a new model from scratch using the same architecture but with the soft labels (and with the same temperature as before).


In this notebook, you will implement the final 2 steps and evaluate the results.

In [1]:
# IF YOU ARE IN COLAB OR HAVE NOT INSTALLED `xlab-security`
!pip install xlab-security # should not take more than a minute or two to run

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 4.5 MB/s eta 0:00:00


In [2]:
from huggingface_hub import hf_hub_download

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Flatten, Linear, ReLU
from torch.utils.data import DataLoader, TensorDataset
import xlab

device = xlab.utils.get_best_device()

### Preliminaries: Train an image classifer on hard labels

We have already completed this step for you. We trained a simple MLP on the MNIST dataset on for two epochs and achieved a 94.90% accuracy on the test set. Importantly, we use a softmax with temperature ($T=20$) as described in our [explainer page](https://xlabaisecurity.com/adversarial/defensive-distillation/).


If interested you can see the output of our training run [here](https://github.com/zroe1/xlab-ai-security/blob/main/models/defensive_distillation/training_output.txt) and the complete code [here](https://github.com/zroe1/xlab-ai-security/tree/main/models/defensive_distillation). You will train your own version of this model for step 5 of this notebook.



In [3]:
# skeleton of the model we trained
class FeedforwardMNIST(nn.Module):
    """Simple 4-layer MLP for MNIST classification"""

    def __init__(self, num_classes=10):
        super(FeedforwardMNIST, self).__init__()

        input_size = 28 * 28
        self.fc1 = Linear(input_size, 256)
        self.fc2 = Linear(256, 64)
        self.fc3 = Linear(64, num_classes)

        self.flatten = Flatten()
        self.relu = ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model_path = hf_hub_download(
    repo_id="uchicago-xlab-ai-security/base-mnist-model",
    filename="mnist_mlp_temp_30.pth",
)
model = torch.load(model_path, map_location=device, weights_only=False)

mnist_mlp_temp_30.pth:   0%|          | 0.00/877k [00:00<?, ?B/s]

In [4]:
print(model)

FeedforwardMNIST(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
)


### Benchmark on PGD

We will benchmark on the pretained model you just loaded. Note that the model already has most of the resistence to adversarial attacks that you will see in this notebook. This is because we trained the model with a temperature greater than one which already accomplishes most of the smoothing. For comparison, the end of the notebook includes code for loading and benchmarking a model trained with a temperature of one, which you will see has almost 0% robustness against 100 iterations of PGD.

When you train your distilled model you should only see a small reduction in attack success. This is actually expected! The authors of the original paper note that the distilled model should in theory converge to the original model but emperically it can offer some additional protection.

If the original model is responsible for most of the protection you may wonder why we don't have you implement it. The reason we don't have you train the original model in this notebook is because it is extremely similar to what you will do in step 4. If you are interested, you should find it fairly easy to replace our pretrained model with your own implementation.



In [5]:
num_test_imgs = 100
imgs, ys = xlab.utils.load_mnist_test_samples(num_test_imgs)
loss_fn = torch.nn.CrossEntropyLoss()

num_success = 0

for img, y in zip(imgs, ys):
    adv_x = xlab.utils.PGD(
        model, loss_fn, img, y, epsilon=12 / 255, alpha=2 / 255, num_iters=20
    )
    adv_y = torch.argmax(model(adv_x))

    if adv_y.item() != y:
        num_success += 1

print(f"{(num_success / num_test_imgs) * 100:.4}% of attacks succeded")

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.99MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 158kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.93MB/s]


29.0% of attacks succeded


## Task #1 and #2: Create new training set

We will be training our distilled model on the labels of the pretrained model you have loaded above.


The model you loaded however, gives logits, not a temperature-smoothed softmax, so to get the proper labels, you will first have to implement the function below which returns softmax with temperature.


<details>
<summary>🔐 <b>Solution for Task #1</b></summary>

```python
def softmax_with_temp(inputs, T):
    """Applies temperature-scaled softmax to inputs
    Args:
        inputs [batch, features]: Input logits tensor.
        T (float): Temperature scaling parameter.
    Returns:
        [batch, features]: Temperature-scaled softmax probabilities.
    """
    out = inputs / T
    return F.softmax(out, dim=1)
```
</details>

In [8]:
def softmax_with_temp(inputs, T):
    """Applies temperature-scaled softmax to inputs
    Args:
        inputs [batch, features]: Input logits tensor.
        T (float): Temperature scaling parameter.
    Returns:
        [batch, features]: Temperature-scaled softmax probabilities.
    """
    return F.softmax(inputs / T, dim=1)

In [9]:
_ = xlab.tests.distillation.task1(softmax_with_temp)


Running tests for Defensive Distillation, Task 1...
🎉 All tests passed! (5/5)

Detailed output:
============================= test session starts ==============================
collecting ... collected 13 items / 8 deselected / 5 selected

../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_softmax_with_temp_implementation[input_tensor0-1.0-basic functionality, T=1.0] PASSED [ 20%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_softmax_with_temp_implementation[input_tensor1-2.0-multi-batch, T=2.0] PASSED [ 40%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_softmax_with_temp_implementation[input_tensor2-0.5-temperature sharpening, T=0.5] PASSED [ 60%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_softmax_with_temp_implementation[input_tensor3-1.0-edge case: all zeros] PASSED [ 80%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_softmax_properties PASS

Now you will find the labels for each batch by calling the model and running it's outputs through `softmax_with_temp`.

<details>
<summary>🔐 <b>Solution for Task #2</b></summary>

```python
def get_batch_labels(batch, T):
    """Generates temperature-scaled probability distributions for a batch
    Args:
        batch [batch, *]: Input batch tensor.
        T (float): Temperature scaling parameter.
    Returns:
        [batch, num_classes]: Temperature-scaled softmax probabilities.
    """
    outs = model(batch)
    outs = softmax_with_temp(outs, T)
    return outs
```
</details>

In [10]:
def get_batch_labels(batch, T):
    """Generates temperature-scaled probability distributions for a batch
    Args:
        batch [batch, *]: Input batch tensor.
        T (float): Temperature scaling parameter.
    Returns:
        [batch, num_classes]: Temperature-scaled softmax probabilities.
    """
    outputs = model(batch)
    return softmax_with_temp(inputs=outputs, T=T)

In [11]:
_ = xlab.tests.distillation.task2(get_batch_labels, model)


Running tests for Defensive Distillation, Task 2...
🎉 All tests passed! (4/4)

Detailed output:
============================= test session starts ==============================
collecting ... collected 13 items / 9 deselected / 4 selected

../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_get_batch_labels_basic[1-1.0-single sample, T=1.0] PASSED [ 25%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_get_batch_labels_basic[3-2.0-small batch, T=2.0] PASSED [ 50%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_get_batch_labels_basic[2-0.5-batch of 2, T=0.5] PASSED [ 75%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_get_batch_labels_consistency PASSED [100%]

=============================== warnings summary ===============================
../usr/local/lib/python3.12/dist-packages/_pytest/config/__init__.py:1290
../usr/local/lib/python3.12/dist-packages/_pytest/config/__init__.py:1

In [12]:
train_loader = xlab.utils.get_mnist_train_loader(batch_size=128, shuffle=True)

In [13]:
imgs = []
soft_labels = []

with torch.no_grad():
    for x_batch, _ in train_loader:
        x_batch = x_batch.to(device)
        soft_labels_batch = get_batch_labels(x_batch, 30)

        imgs.append(x_batch.cpu())
        soft_labels.append(soft_labels_batch.cpu())

all_images = torch.cat(imgs, dim=0)
all_soft_labels = torch.cat(soft_labels, dim=0)
soft_label_dataset = TensorDataset(all_images, all_soft_labels)

batch_size = 128
soft_label_loader = DataLoader(
    soft_label_dataset,
    batch_size=batch_size,
    shuffle=True,
)

The first step in contructing this new dataset is to implement `get_batch_labels` by calling the pretrained model with temperature T.

## Task #3 and #4: Train distilled model

In [15]:
# skeleton of the model we trained
distilled = FeedforwardMNIST().to(device)

The optimization problem from the original paper was formalized by the authors using the following equation:

$$
\arg\min_{\theta_F} -\frac{1}{|\mathcal{X}|} \sum_{X \in \mathcal{X}} \sum_{i \in 0..N} F_i(X) \log F_i^d(X)
$$

The loss for a single example is simply cross entropy loss with soft labels:

$$
\mathcal{L}(X) = -\sum_{i \in 0..N} F_i(X) \log F_i^d(X)
$$

<details>
<summary>🔐 <b>Solution for Task #3</b></summary>

```python
def cross_entropy_loss_soft(soft_labels, probs):
    """Computes cross-entropy loss between soft labels and predicted probabilities
    Args:
        soft_labels [batch, num_classes]: Target probability distributions.
        probs [batch, num_classes]: Predicted probability distributions.
    Returns:
        scalar tensor: Normalized cross-entropy loss value.
    """
    assert soft_labels.shape == probs.shape
    batch_size = soft_labels.shape[0]

    log_probs = torch.log(probs)
    return torch.sum(-1 * log_probs *  soft_labels) / batch_size
```
</details>

In [16]:
def cross_entropy_loss_soft(soft_labels, probs):
    """Computes cross-entropy loss between soft labels and predicted probabilities
    Args:
        soft_labels [batch, num_classes]: Target probability distributions.
        probs [batch, num_classes]: Predicted probability distributions.
    Returns:
        scalar tensor: Normalized cross-entropy loss value.
    """

    assert soft_labels.shape == probs.shape
    batch_size = soft_labels.shape[0]

    log_probs = torch.log(probs)
    return (-1 / batch_size) * torch.sum(log_probs * soft_labels)

In [17]:
_ = xlab.tests.distillation.task3(cross_entropy_loss_soft)


Running tests for Defensive Distillation, Task 3...
🎉 All tests passed! (4/4)

Detailed output:
============================= test session starts ==============================
collecting ... collected 13 items / 9 deselected / 4 selected

../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_cross_entropy_loss_soft_implementation[1-3-single sample, 3 classes] PASSED [ 25%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_cross_entropy_loss_soft_implementation[5-10-small batch, 10 classes] PASSED [ 50%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_cross_entropy_loss_soft_implementation[2-5-batch of 2, 5 classes] PASSED [ 75%]
../usr/local/lib/python3.12/dist-packages/xlab/tests/distillation.py::test_cross_entropy_loss_one_hot_labels PASSED [100%]

=============================== warnings summary ===============================
../usr/local/lib/python3.12/dist-packages/_pytest/config/__init__.py:1290
../usr/lo

Now you will fill in the function to train your distilled model. Most of this work has already been done for you.

Note that there are no tests for this task. You can evaluate the quality of your solution by benchmarking your model in the following section.

<details>
<summary>🔐 <b>Solution for Task #4</b></summary>

```python
def train(model, epochs, train_loader, T):
    """Trains model using soft label cross-entropy loss with temperature scaling
    Args:
        model: Neural network model to train.
        epochs (int): Number of training epochs.
        train_loader: DataLoader providing batches of images and soft labels.
        T (float): Temperature scaling parameter for softmax.
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

    for epoch in range(epochs):
        for i, (img, soft_label) in enumerate(train_loader):
            optimizer.zero_grad()

            # 1. get logits from model
            img, soft_label = img.to(device), soft_label.to(device)
            logits = model(img)

            # 2. process the logits with softmax_with_temp
            out = softmax_with_temp(logits, T)

            # 3. compute batch loss
            batch_loss = cross_entropy_loss_soft(soft_label, out)
    
            if i % 50==0:
                print(f"Epoch #{epoch + 1}: batch loss = {batch_loss.item():.4f}")
    
            batch_loss.backward()
            optimizer.step()
```
</details>

In [18]:
def train(model, epochs, train_loader, T):
    """Trains model using soft label cross-entropy loss with temperature scaling
    Args:
        model: Neural network model to train.
        epochs (int): Number of training epochs.
        train_loader: DataLoader providing batches of images and soft labels.
        T (float): Temperature scaling parameter for softmax.
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

    for epoch in range(epochs):
        for i, (img, soft_label) in enumerate(train_loader):
            optimizer.zero_grad()

            ######### YOUR CODE STARTS HERE #########
            # 1. get logits from model
            logits = model(img.to(device))
            # 2. process the logits with softmax_with_temp
            output = softmax_with_temp(inputs=logits, T=T)
            # 3. compute batch loss
            batch_loss = cross_entropy_loss_soft(
                soft_labels=soft_label.to(device), probs=output
              )
            ########## YOUR CODE ENDS HERE ##########

            if i % 50 == 0:
                print(f"Epoch #{epoch + 1}: batch loss = {batch_loss.item():.4f}")

            batch_loss.backward()
            optimizer.step()

In [19]:
train(distilled, 3, soft_label_loader, 30)

Epoch #1: batch loss = 2.3027
Epoch #1: batch loss = 0.5587
Epoch #1: batch loss = 0.4124
Epoch #1: batch loss = 0.3124
Epoch #1: batch loss = 0.3159
Epoch #1: batch loss = 0.3410
Epoch #1: batch loss = 0.2867
Epoch #1: batch loss = 0.3068
Epoch #1: batch loss = 0.2677
Epoch #1: batch loss = 0.2929
Epoch #2: batch loss = 0.2097
Epoch #2: batch loss = 0.2531
Epoch #2: batch loss = 0.2784
Epoch #2: batch loss = 0.3137
Epoch #2: batch loss = 0.2115
Epoch #2: batch loss = 0.2471
Epoch #2: batch loss = 0.2792
Epoch #2: batch loss = 0.2309
Epoch #2: batch loss = 0.2979
Epoch #2: batch loss = 0.2718
Epoch #3: batch loss = 0.2642
Epoch #3: batch loss = 0.2192
Epoch #3: batch loss = 0.2431
Epoch #3: batch loss = 0.2130
Epoch #3: batch loss = 0.2629
Epoch #3: batch loss = 0.2325
Epoch #3: batch loss = 0.2623
Epoch #3: batch loss = 0.2723
Epoch #3: batch loss = 0.2427
Epoch #3: batch loss = 0.2512


## Benchmarking our Defense

Below you should see that the clean accuracy is comparable to the original 94.90% accuracy. The attack success rate should be a bit below the success rate of the pretrained model. As we explained above, a lot of the protection comes from the original temperature smoothing, so you should not be surprised if the success rate is only slightly below the original pretrained model.

In [20]:
clean_acc = xlab.utils.evaluate_mnist_accuracy(distilled)
print(f"Clean accuracy of distilled model: {clean_acc * 100:.2f}%")

Clean accuracy of distilled model: 93.96%


In [21]:
num_test_imgs = 100
imgs, ys = xlab.utils.load_mnist_test_samples(num_test_imgs)
loss_fn = torch.nn.CrossEntropyLoss()

num_success = 0

for img, y in zip(imgs, ys):
    adv_x = xlab.utils.PGD(
        distilled, loss_fn, img, y, epsilon=12 / 255, alpha=2 / 255, num_iters=20
    )
    adv_y = torch.argmax(distilled(adv_x))

    if adv_y.item() != y:
        num_success += 1

print(f"{(num_success / num_test_imgs) * 100:.4}% of attacks succeded")

32.0% of attacks succeded


In [22]:
model_path = hf_hub_download(
    repo_id="uchicago-xlab-ai-security/base-mnist-model",
    filename="mnist_mlp_temp_1.pth",
)
standard = torch.load(model_path, map_location=device, weights_only=False)

mnist_mlp_temp_1.pth:   0%|          | 0.00/877k [00:00<?, ?B/s]

## Benchmarking a Traditional Model

For reference, below you will the clean accuracy and attack success rate of a model with the same architecture trained with a softmax temperature of 1.

In [23]:
clean_acc = xlab.utils.evaluate_mnist_accuracy(standard)
print(f"Clean accuracy of standard model: {clean_acc * 100:.2f}%")

num_test_imgs = 30
imgs, ys = xlab.utils.load_mnist_test_samples(num_test_imgs)
loss_fn = torch.nn.CrossEntropyLoss()

num_success = 0

for img, y in zip(imgs, ys):
    adv_x = xlab.utils.PGD(
        standard, loss_fn, img, y, epsilon=12 / 255, alpha=2 / 255, num_iters=10
    )
    adv_y = torch.argmax(standard(adv_x))

    if adv_y.item() != y:
        num_success += 1

print(f"{(num_success / num_test_imgs) * 100:.4}% of attacks succeded")

Clean accuracy of standard model: 96.62%
40.0% of attacks succeded
